<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/SLC_AlgFairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 9: Algorithmic Bias and Fairness

Dani Bauer (w/ Jim Guszcza), 11/2023

In this tutorial, we discuss approaches how to analyze a predictive algorithm with regards to "fairness." We do so in the context of a well-known case study on an algorithm that assists judges in parole decisions. We go over different notions of fairness, discuss tradeoffs, and explain the intuition behind the results of the analyses. We also discuss approaches how to adjust algorithms to enforce fairness. A second case study, which we will ask you to work on, illustrates how these ideas apply in the actuarial context.

### Load required packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_curve, auc

In [ ]:
!pip install aequitas
#Another library that seems to be popular is fariness 360:
#!pip install aif360

In [3]:
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

# Compas Case Study

The Compas data originates from a [well-known case study on algorithmic bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing). The background is [as follows](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm): Across the nation, judges, probation and parole officers are increasingly using algorithms to assess a criminal defendant's likelihood of becoming a recidivist---a term used to describe criminals who re-offend. One of the commercial tools made by Northpointe, Inc. is called COMPAS (which stands for Correctional Offender Management Profiling for Alternative Sanctions). The case study compares outcomes and risk scores for individuals belonging to different races. In what follows, we will go through some of these analyses ourselves.

## The Compas Data

The data is in our github folder. Let's take a look:

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [ ]:
dat = pd.read_csv('ML_656/compasdata.csv')
dat.head()

In [ ]:
dat.describe()

The data contains information on recidivism of 6,172 individuals as well as information on the individual's age, sex, criminal history, their ethnicity---and the risk score they received from the COMPAS algorithm.

To simplify the situation, we focus on two ethnicity levels only: African-Americans and Caucasians.

In [ ]:
dat = dat.loc[(dat['ethnicity'] == 'Caucasian') | (dat['ethnicity'] == 'African_American')]
dat.describe()

So we still have the majority of individuals included. We will consider the African-Americans as the "protected" class.

We commence by exploring the data some and looking at fairness manually, but then we will also explore how to use the Aequitas package in this setting.

### Fairness scores vs. recidivism rates

Let's start by comparing the COMPAS scores between the two ethic groups (see also the density plots from the fairness package above):

In [ ]:
plt.title("Score distribution by group")
plt.xlabel("Group: 1 = Caucasian, 2 = African-American")
plt.boxplot([dat.loc[dat['ethnicity'] == 'Caucasian']['probability'],dat.loc[dat['ethnicity'] == 'African_American']['probability']])
plt.show()

The distribution of scores in the African-American group has a higher median and higher percentiles than the scores in the Caucasian group.

However, consider the number of re-offenders between the group, we see the following:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
by_race = sns.countplot(x="ethnicity", hue="Two_yr_Recidivism", data=dat[dat.ethnicity.isin(['African_American', 'Caucasian'])], palette=aq_palette)

Let's consider raw averages of re-offenders in the two groups -- for caucasians:

In [ ]:
np.average(dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == 'yes')

And for the protected group:

In [ ]:
np.average(dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == 'yes')

So, we can interpret the higher average COMPAS scores as resulting from statistical facts in this particular population.

Let’s investigate accuracy, i.e., how accurate the algorithm is, on aggregate and by protected vs. non-protected class. In doing so, we determine the correctly classified re-offenders (true positives), the correctly cassified non-re-offenders (true negatives), the incorrectly classified re-offenders (false negatives), and the incorrectly classified non-re-offenders (false positives)—by class.

In [ ]:
TP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
TP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * dat.loc[dat['ethnicity'] == 'African_American']['predicted'])
FP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
FP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * dat.loc[dat['ethnicity'] == 'African_American']['predicted'])
FN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
FN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * (dat.loc[dat['ethnicity'] == 'African_American']['predicted'] == 0))
TN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
TN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * (dat.loc[dat['ethnicity'] == 'African_American']['predicted'] == 0))

"Accuracy" now is simply the correctly classified individuals divided by all individuals.

In [ ]:
acc = (TP_0+TP_1+TN_0+TN_1)/(TP_0 + FP_0 + TP_1 + FP_1 + TN_0 + FN_0 + TN_1 + FN_1)
acc

and by group:

In [ ]:
acc_0 = (TP_0+TN_0)/(TP_0 + FP_0 + TN_0 + FN_0)
acc_0

and for the protected group:

In [ ]:
acc_1 = (TP_1+TN_1)/(TP_1 + FP_1 + TN_1 + FN_1)
acc_1

In [ ]:
plt.bar(['Caucasian','African-American'], [acc_0,acc_1], color ='maroon', width = 0.4)
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.title("Accuracy:    (TP+TN) / (TP+FP+TN+FN)")
plt.show()

It appears that the algorithm is similarly "accurate" for both groups, and in fact the accuracy is even somewhat higher for the protected group.

Hence, one may conclude that the algorithm performs reasonably well---and that there are no major concerns with regards to differential performance for the protected group. In fact, this may have been the mindset by the creators of the COMPAS algorithm.

### Alternative Fairness Metrics

While "accuracy"---or one minus accuracy, which is the misclassification rate---do not point to problems, we can investigate more differentiated error metrics. One obvious way of doing this is by comparing

We can further assess the performance by visualizing false negative rates and the true positive rates by group. One way of looking at this is how many of the negatively classified individuals (low risk) were labeled so incorrectly, and how many of the positively classified (high risk) were labeled so correctly. The former proportion of negatively classified for which outcome indeed was one is also referred to as the *False Omission Rate*, whereas the latter proportion of positively classified that were indeed correct is also referred to as *Precision*:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['False Omission Rate',FN_0/(TN_0+FN_0),'Caucasian'],['False Omission Rate',FN_1/(TN_1+FN_1),'African_American'],['Precision',TP_0/(TP_0+FP_0),'Caucasian'],['Precision',TP_1/(TP_1+FP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette)

We start seeing some differences between the groups, although the differences are not startling. The algorithm seems to me more “precise” for the protected class.

Differences between the groups start to emerge if we consider indviduals that are labeled as being likely to re-offend, i.e., the total fraction of positives between the two groups:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['Proportional Parity:  (FP+TP) / (FP+TP+TN+FN)',(FP_0+TP_0)/(TN_0+FN_0+FP_0+TP_0),'Caucasian'],['Proportional Parity:  (FP+TP) / (FP+TP+TN+FN)',(FP_1+TP_1)/(TN_1+FN_1+FP_1+TP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette).set(title='proportion of group classified positive -- a measure of disparate impact')

Hence, it appears that the COMPAS algorithm does have disparate impact between the two classes. Based on our comparisons from above, one may argue that this is a consequence of the statistical fact that within this population, the rate of re-offenders in the protected class was higher. Indeed, the disparate impact is associated with the accuracy of the algorithm.

In relation to the disparate impact, we can ask how the algorithm performs with regards to accurately labeling individuals. We start by investigating true positives, i.e., the proportion of positives that were classified correctly—which is also called the sensitivity. One minus the sensitivity corresponds to the positives that were incorrectly labeled, i.e., the false negative rate:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['Sensitivity (aka TPR):   TP / (TP + FN)',TP_0 / (FN_0+TP_0),'Caucasian'],['Sensitivity (aka TPR):   TP / (TP + FN)',TP_1 / (FN_1+TP_1),'African_American'],['False negative rate:  FN/(FN+TP)',FN_0 / (FN_0+TP_0),'Caucasian'],['False negative rate:  FN/(FN+TP)',FN_1 / (FN_1+TP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette)

The relationship between the sensitivity and the false negative rate is:

In [ ]:
TP_0 / (FN_0+TP_0) + FN_0 / (FN_0+TP_0)

In [ ]:
TP_1 / (FN_1+TP_1) + FN_1 / (FN_1+TP_1)

So it appears that the COMPAS algorithm is more "sensitive" for the protected class, and, ergo, the false negative rate is higher for the non-protected class. The latter means that the the number of high-risk individuals that were missed---and erroneously recommended for parole---is higher for the non-protected class.

Recall that *Sensitivity* is on the y-axis of the ROC curve, so we know there is a tradeoff between sensitivity and the *False Positive Rate* (FPR), which in turn is one minus *Specificity*. The FPR simply is the percent of negatives incorrecly labeled as one:

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['False positive rate:  FP/(FP+TN)',FP_0 / (FP_0+TN_0),'Caucasian'],['False positive rate:  FP/(FP+TN)',FP_1 / (FP_1+TN_1),'African_American'],['Specificity:   TN / (TN + FP)',TN_0 / (TN_0+FP_0),'Caucasian'],['Specificity:   TN / (TN + FP)',TN_1 / (TN_1+FP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette)

The relationship between the specificity and the false positive rate is:

In [ ]:
FP_0 / (FP_0+TN_0) + TN_0 / (TN_0+FP_0)

In [ ]:
FP_1 / (FP_1+TN_1) + TN_1 / (TN_1+FP_1)

We notice that the FPR is substantially higher for the protected class, i.e., the proportion of individuals that were erroneously labeled as high risk to re-offend is substantially higher in the protected class than in the non-protected class! Being more at risk of being erroneously labeled as high-risk to re-offend based on the color of risk does seem like a problem.

### Intution: Generalized Confusion Matrix

But how is it possible that the algorithm appears similarly accurate for the two groups, yet the chance of being erroneously classified as high risk is much higher for the protected group?

To provide intuition, let's generate a generalized confusion table, where we illustrate all the correctly and erronrously classified individuals by class:


In [ ]:
data_tmp = [['prot=0 event=0',TN_0,FP_0],['prot=0 event=1',FN_0,TP_0],['prot=1 event=0',TN_1,FP_1],['prot=1 event=1',FN_1,TP_1]]
df_tmp = pd.DataFrame(data_tmp, columns=[' ', 'decision=0','decision=1'])
df_tmp

And let’s visualize:

In [ ]:
x = ['decision=0','decision=1']
y1 = np.array([FN_1,TP_1])
y2 = np.array([TN_1,FP_1])
y3 = np.array([FN_0,TP_0])
y4 = np.array([TN_0,FP_0])

plt.bar(x, y1, color='r')
plt.bar(x, y2, bottom=y1, color='b')
plt.bar(x, y3, bottom=y1+y2, color='y')
plt.bar(x, y4, bottom=y1+y2+y3, color='g')
plt.ylabel("Count")
plt.legend(["prot=1 event=1", "prot=1 event=0", "prot=0 event=1", "prot=0 event=0"])
plt.title("Scores by Teams in 4 Rounds")
plt.show()

We have the labels on the horizontal, and the actual outcomes on the vertical. The large FPR is driven by the large fraction of individuals in the protected class labeled as one.

### Enforcing Fairness via Different Thresholds

As we illustrated, there are different notions of fairness one may apply. However, it seems intutive that the high FPR for the protected class may be perceived problematic. Being erroneously denied parole based on the color of one's skin arguably is perceived as unfair by many.

This intuition is in line with prescriptions from scholars that have attempted to organize different fairness metrics. For instance, the bias audit toolkit [Aequitas](http://www.datasciencepublicpolicy.org/our-work/tools-guides/aequitas/) provides a "fairness tree" that organizes different metrics. Considering this situation where interventions are "punitive" and we are worried about individuals where "intervention is not warranted" (i.e., those that won't re-offend), Aequitas' fairness tree suggests that "FPR Parity"---i.e., equating FPR rates across the different classes---is an appropriate metric.

How can we realize this? We can adjust the threshold for the protected class to above 0.5, i.e., requiring a higher score to be labeled high risk. After some experimentation, it appears 0.605 is a threshold that roughly equates false positive rates:

In [ ]:
upd_afram = (dat.loc[dat['ethnicity'] == 'African_American']['probability'] > 0.605)

In [ ]:
TP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
TP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * upd_afram)
FP_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'])
FP_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * upd_afram)
FN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "yes") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
FN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "yes") * (upd_afram == 0))
TN_0 = sum((dat.loc[dat['ethnicity'] == 'Caucasian']['Two_yr_Recidivism'] == "no") * (dat.loc[dat['ethnicity'] == 'Caucasian']['predicted'] == 0))
TN_1 = sum((dat.loc[dat['ethnicity'] == 'African_American']['Two_yr_Recidivism'] == "no") * (upd_afram == 0))

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
data_tmp = [['False positive rate:  FP/(FP+TN)',FP_0 / (FP_0+TN_0),'Caucasian'],['False positive rate:  FP/(FP+TN)',FP_1 / (FP_1+TN_1),'African_American'],['Specificity:   TN / (TN + FP)',TN_0 / (TN_0+FP_0),'Caucasian'],['Specificity:   TN / (TN + FP)',TN_1 / (TN_1+FP_1),'African_American']]
df_tmp = pd.DataFrame(data_tmp, columns=['Metric','rate','ethnicity'])
sns.barplot(df_tmp, x="Metric", y="rate", hue="ethnicity", palette=aq_palette)

Adjusting the threshold generally results in a less "accurate" algorithm. Let's check the accuracy level for the modified algorithm:

In [ ]:
acc = (TP_0+TP_1+TN_0+TN_1)/(TP_0 + FP_0 + TP_1 + FP_1 + TN_0 + FN_0 + TN_1 + FN_1)
acc

Surprisingly, the accuracy only drops slightly from the original accuracy score of 0.667.

For the non-protected group it stays the same:

In [ ]:
acc_0 = (TP_0+TN_0)/(TP_0 + FP_0 + TN_0 + FN_0)
acc_0

But there is a decrease for the protected class:

In [ ]:
acc_1 = (TP_1+TN_1)/(TP_1 + FP_1 + TN_1 + FN_1)
acc_1

In [ ]:
plt.bar(['Caucasian','African-American'], [acc_0,acc_1], color ='maroon', width = 0.4)
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.title("Accuracy:    (TP+TN) / (TP+FP+TN+FN)")
plt.show()

So we obtain a slight drop for the protected group, but this appears a small price to pay to ensure the equivalence of false positive rates and the associated fairness in this example setting.

## Using the Aequitas Library

Let's now see how to use Aequitas library to analyze the predictions for fairness. More details can be found [here](https://colab.research.google.com/github/dssg/aequitas/blob/update_compas_notebook/docs/source/examples/compas_demo.ipynb).

### Data Format

Aequitas requires a column named `score` that has the predicted label by the algorithm (should be 0 / 1) and a column named `label_value` that has the actual class. We reformat our dataset to adhere to this format, and we also drop all the non-ethnicity columns to focus on that attribute. (One needs to drop the non-string columns in any case, otherwise there is an error message.)

In [ ]:
dat = dat.rename(columns={"predicted": "score", "Two_yr_Recidivism": "label_value"})
dat = dat.drop(columns=['Number_of_Priors', 'probability','Unnamed: 0','Age_Above_FourtyFive','Age_Below_TwentyFive','Female','Misdemeanor'])
dat.label_value = dat.label_value.map(dict(yes=1, no=0))
dat.head()

### Using Aequitas to Obtain Confusion Values and Rates

We can now use Aequitas to obtain the count of true positives, false positives, true negatives, and false negatives by group--as well as corresponding rates.

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(dat)

In [ ]:
absolute_metrics = g.list_absolute_metrics(xtab)

In [ ]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

### Bias Analyses

We can now obtain bias outcomes, where we have to define a reference group. Here we use the Caucasian subgroup.

In [ ]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=dat,
                                        ref_groups_dict={'ethnicity':'Caucasian'},
                                        alpha=0.05, check_significance=True,
                                        mask_significance=True)
bdf.style

We can use a variety of metrics. Given the discussion above, we use false positive rate (FPR):

In [ ]:
metrics = ['fpr']
disparity_tolerance = 1.25

Let's visualize:

In [ ]:
ap.summary(bdf, metrics, fairness_threshold = disparity_tolerance)

In [ ]:
ap.disparity(bdf, metrics, 'ethnicity', fairness_threshold = disparity_tolerance)

As we have seen above, the FPR is substantially higher for the African-American poppulation--and also for native Americans.